In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import mapboxgl
from mapboxgl.viz import*
from mapboxgl.utils import *
from config import token
from mapboxgl.utils import create_color_stops, df_to_geojson
from mapboxgl.utils import df_to_geojson
from pyproj import Proj, transform

In [8]:
df = pd.read_excel('df.xlsx', index_col = 0)
df = df.drop(['관정깊이(m)', '자연수위(dtw, m)', '토지이용'], axis=1)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5732 entries, ㈜투건 한라산C. to nan
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   X          5278 non-null   object 
 1   Y          5278 non-null   object 
 2   elevation  5102 non-null   float64
 3   year       5732 non-null   int64  
 4   month      5459 non-null   float64
 5   EC         5685 non-null   float64
 6   pH         5188 non-null   object 
 7   HCO3       5148 non-null   float64
 8   Cl         5634 non-null   object 
 9   NO3-N      5642 non-null   object 
 10  SO4        5634 non-null   object 
 11  Na         4999 non-null   object 
 12  K          4999 non-null   object 
 13  Mg         5000 non-null   object 
 14  Ca         5000 non-null   float64
dtypes: float64(5), int64(1), object(9)
memory usage: 716.5+ KB


In [10]:
parameters = ['X','Y','pH','Cl','NO3-N','SO4','Na','K','Mg']
df[parameters] = df[parameters].apply(pd.to_numeric, errors='coerce', axis=1)

In [11]:
df = df.dropna(subset=['X', 'Y', 'elevation', 'year', 'month', 'EC', 'pH', 'HCO3', 'Cl', 'NO3-N', 'SO4', 'Na', 'K', 'Mg', 'Ca'])

In [12]:
out_location = df[df['elevation'] == 179].index
out_location2 = df[(df['elevation']==80) & (df['year']==2021)].index
df = df.drop(out_location)
df = df.drop(out_location2)

In [13]:
proj_TM = Proj(init='epsg:5175')
proj_WGS84 = Proj(init='epsg:4326')

def transform_TM_to_w84(df):
    return pd.Series(transform(proj_TM, proj_WGS84, df['X'], df['Y']), index=['X','Y'])

df[['X', 'Y']]= df.apply(transform_TM_to_w84, axis=1)

In [14]:
geo_data = df_to_geojson(df=df, lat='Y', lon='X')

color_breaks = [0, 5, 10, 20, 40, 100, 400]
color_stops = create_color_stops(color_breaks, colors='BuPu')

center = [126.566, 33.352]

viz = CircleViz(access_token= token,
                data=geo_data,
                height='800px',
                color_property = "Cl",
                color_stops = color_stops,
                style='mapbox://styles/mapbox/dark-v10',
                center = center,
                zoom = 9.5,
                below_layer = 'waterway-label')

viz.show()